### Building evasion vs. mean time interval : by Bus by Service

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from importlib import reload
import datetime as dt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
from Utils import TransantiagoConstants

In [ ]:
first_quarter_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/06_RFA/01_EvasionTrimestral/01_analisis/1st_quarter.xlsx"
second_quarter_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/06_RFA/01_EvasionTrimestral/01_analisis/2nd_quarter.xlsx"
third_quarter_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/06_RFA/01_EvasionTrimestral/01_analisis/3rd_quarter.xlsx"
codes_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/04_DTPM/codes_services.xlsx"

In [ ]:
first_quarter_evasion = pd.read_excel(first_quarter_path, encoding = 'latin-1')
second_quarter_evasion = pd.read_excel(second_quarter_path, encoding = 'latin-1')
third_quarter_evasion = pd.read_excel(third_quarter_path, encoding = 'latin-1')
codes = pd.read_excel(codes_path, encoding = 'latin-1')

In [ ]:
first_quarter_evasion['TIEMPO'] = first_quarter_evasion['HORA'].astype(str)+':'+first_quarter_evasion['MINUTOS'].astype(str)+':00'
second_quarter_evasion['TIEMPO'] = second_quarter_evasion['HORA'].astype(str)+':'+second_quarter_evasion['MINUTOS'].astype(str)+':00'
third_quarter_evasion['TIEMPO'] = third_quarter_evasion['HORA'].astype(str)+':'+third_quarter_evasion['MINUTOS'].astype(str)+':00'

In [ ]:
first_quarter_evasion.columns = ['FECHA', 'SERVICIO','PATENTE','PUERTAS','N_PUERTA','LUGAR_INICIO','HORA_INICIO','HORA','MINUTO','INGRESAN','NO_VALIDAN','TP','TIEMPO']
second_quarter_evasion.columns = ['FECHA', 'SERVICIO','TIPO','PATENTE','PUERTAS','N_PUERTA','LUGAR_INICIO','HORA_INICIO','HORA','MINUTO','INGRESAN','NO_VALIDAN','TP','TIEMPO']
third_quarter_evasion.columns = ['FECHA', 'SERVICIO','PATENTE','PUERTAS','N_PUERTA','LUGAR_INICIO','HORA_INICIO','HORA','MINUTO','INGRESAN','NO_VALIDAN','TP','TIEMPO']
del second_quarter_evasion['TIPO']

In [ ]:
type(first_quarter_evasion.loc[0,'TIEMPO'])

In [ ]:
type(first_quarter_evasion.loc[0,'FECHA'])

In [ ]:
frames = [first_quarter_evasion, second_quarter_evasion, third_quarter_evasion]
evasion = pd.concat(frames, keys=['first', 'second', 'third'])

* Processing codes before merging. Merge should be made only by codes_ida or codes_ret. Be aware of it

In [ ]:
codes_ida = codes[codes['DIRECTION']=='Ida']
codes_ret = codes[codes['DIRECTION']=='Ret']

codes_ida = codes_ida.rename(columns = {'USER_CODE':'SERVICIO'})
codes_ret = codes_ret.rename(columns = {'USER_CODE':'SERVICIO'})

del codes_ida['DIRECTION']
del codes_ret['DIRECTION']

In [ ]:
evasion = pd.merge(evasion,codes_ida, on=['SERVICIO'], how='left')

In [ ]:
evasion.loc[evasion['TS_CODE'].isnull(),'SERVICIO'].unique()

* THIS IS HARDCODED

In [ ]:
evasion.loc[evasion['TS_CODE'].isnull(),'TS_CODE']='446'

In [ ]:
evasion.head()

* Processing evasion-ddbb before merging

In [ ]:
evasion['PATENTE'] =  evasion['PATENTE'].str.replace(' ','')

evasion['SERVICIO_TMP'] = evasion['SERVICIO'].apply(str)
evasion['TS_CODE_TMP'] = evasion['TS_CODE'].apply(str)

del evasion['SERVICIO']
del evasion['TS_CODE']

evasion = evasion.rename(columns = {'SERVICIO_TMP':'SERVICIO_USUARIO', 'TS_CODE_TMP':'SERVICIO'})
evasion_paradero = evasion[evasion['TP']=='P']
evasion_paradero_first = evasion_paradero[evasion_paradero['N_PUERTA']==1]

In [ ]:
type(evasion['SERVICIO'][0])

* Getting evasion by bus by service

In [ ]:
summary = evasion_paradero_first.groupby(['PATENTE','SERVICIO','FECHA'])['INGRESAN','NO_VALIDAN'].agg(['count','sum'])
summary.columns=['EVASION_COUNT', 'TOTAL_INGRESAN', 'tmp_count' ,'TOTAL_NO_VALIDAN']
del summary['tmp_count']
summary.reset_index(inplace=True)

* Building complete etapas ddbb.

In [ ]:
common_dates = TransantiagoConstants.common_dates

In [ ]:
reduced_etapas = pd.DataFrame()

for date in common_dates:
    base_input_path = 'C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/01_SSH/02_TIMESBYBUSBYSERVICE/01_ORIGINAL/'
    file_name = date + '-meanByBusByService.csv'
    grouped_by_bus = pd.read_csv(base_input_path+file_name, encoding = 'latin-1',sep =";")
    reduced_etapas = pd.concat([reduced_etapas,grouped_by_bus])


* Processing etapas ddbb before merging

In [ ]:
del reduced_etapas['Unnamed: 0']
reduced_etapas = reduced_etapas.rename(columns = {'fecha':'FECHA', 'sitio_subida':'PATENTE', 'count': 'ETAPAS_COUNT', 'servicio_subida' : 'SERVICIO'})
reduced_etapas['FECHA'] = pd.to_datetime(reduced_etapas.FECHA)
reduced_etapas['PATENTE'] = reduced_etapas['PATENTE'].str.replace("-", "")
reduced_etapas['PATENTE'] = reduced_etapas['PATENTE'].str.replace(" ", "")

In [ ]:
reduced_etapas.reset_index(drop=True,inplace=True)

In [ ]:
type(reduced_etapas.loc[0,'FECHA'])

* Merging.

In [ ]:
evasion_vs_mean = pd.merge(summary,reduced_etapas, on=['PATENTE','SERVICIO','FECHA'], how='left')

In [ ]:
len(evasion_vs_mean.index)

* Getting rid of missing values

In [ ]:
clean_evasion_vs_mean = evasion_vs_mean[evasion_vs_mean['mean']>0]

In [ ]:
len(clean_evasion_vs_mean.index)

* Computing evasion rate: should be computed before... a warning is popping up

In [ ]:
clean_evasion_vs_mean.loc[:,'EVASION_RATE'] = clean_evasion_vs_mean['TOTAL_NO_VALIDAN'] / clean_evasion_vs_mean['TOTAL_INGRESAN']

In [ ]:
clean_evasion_vs_mean.loc[:,'fecha_instalacion'] = pd.to_datetime(clean_evasion_vs_mean.fecha_instalacion)

In [ ]:
clean_evasion_vs_mean.loc[:,'SI_TORNIQUETE'] = (clean_evasion_vs_mean['fecha_instalacion'].notnull())

In [ ]:
clean_evasion_vs_mean.loc[:,'SI_2017_TORNIQUETE'] = (clean_evasion_vs_mean['fecha_instalacion']>pd.to_datetime('2017-01-01'))

In [ ]:
clean_evasion_vs_mean_no_turnstile = clean_evasion_vs_mean[(clean_evasion_vs_mean['SI_TORNIQUETE']==False) | 
                                                           ((clean_evasion_vs_mean['SI_TORNIQUETE']==True) & (clean_evasion_vs_mean['FECHA']<=clean_evasion_vs_mean['fecha_instalacion']))]

In [ ]:
len(clean_evasion_vs_mean_no_turnstile.index)

In [ ]:
fig_1, ax_1 = plt.subplots()

ax_1.scatter(clean_evasion_vs_mean_no_turnstile['mean'],clean_evasion_vs_mean_no_turnstile['EVASION_RATE'])
ax_1.set_title('Evasion vs. Mean Time - No Turns, By Bus and Service, Original')
ax_1.set_xlabel('Mean Time Interval [s]')
ax_1.set_ylabel('Evasion Rate [%]')

plt.savefig('C:/Users/leoca_000/Desktop/Evasion/03_report/02_Figs/8_Original_EvasionVsMeanTimeByBusByService.pdf')

### Closed